<a href="https://colab.research.google.com/github/profteachkids/CHE5136_Fall2021/blob/main/DataImport_in_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dotmap

In [34]:
import requests
import re
import string
from dotmap import DotMap
import numpy as np

In [13]:
extract_single_props = {'Molecular Weight' : 'Mw',
                        'Critical Temperature' : 'Tc',
                        'Critical Pressure' : 'Pc',
                        'Critical Volume' : 'Vc',
                        'Acentric factor' : 'w',
                        'Normal boiling point' : 'Tb',
                        'IG heat of formation' : 'HfIG',
                        'IG Gibbs of formation' : 'GfIG',
                        'Heat of vaporization' : 'HvapNB'}

extract_coeff_props={'Vapor Pressure' : 'Pvap',
                     'Ideal Gas Heat Capacity':'CpIG',
                     'Liquid Heat Capacity' : 'CpL',
                     'Solid Heat Capacity' : 'CpS',
                     'Heat of Vaporization' : 'Hvap',
                     'Liquid Density' : 'rhoL'}

In [10]:
baseurl = 'https://raw.githubusercontent.com/profteachkids/chetools/main/data/'
suffix = 'Props.txt'
compounds = ['Water']
single_props_pat = re.compile(r'^\s+([\w \/.]+?)\s+:\s+([-.0-9e+]+) +([-\w/.()*]*) *$', re.MULTILINE)
coeffs_name_pat = re.compile(r'([\w ]+)\s[^\n]*Equation.*?Coeffs:([-0-9. e+]*)', re.DOTALL)

In [47]:
text = requests.get(baseurl + 'Methanol' + suffix).text

props={}
single_props=dict([one_prop[:2] for one_prop in single_props_pat.findall(text)])


In [48]:
for k,v in extract_single_props.items():
    props[v]=float(single_props[k])

In [54]:
coeffs_name_strings = dict(coeffs_name_pat.findall(text))
for fullname, abbrevname in extract_coeff_props.items():
    coeff_values = re.split(r'\s+',coeffs_name_strings[fullname])
    for letter, coeff in zip(string.ascii_uppercase,coeff_values[1:]):
        props[abbrevname+letter]=float(coeff)

props=DotMap(props)

In [55]:
def vapor_pressure(T):
    return np.exp(props.PvapA + props.PvapB/T + props.PvapC*np.log(T) + props.PvapD*T**props.PvapE)

In [57]:
vapor_pressure(64.7+273.15)

102201.44791476373

In [53]:
props.PvapA

AttributeError: ignored